In [23]:
numberOfSamples = 10
outputDirectory = r"E:\Nextcloud\Public\FOR2363\TP4\LUH\HVAC-Systems\BatchCurrent\Data_Source\Simulation-Files"
weatherFilePath = r"C:\EnergyPlusV22-2-0\WeatherData\DEU_Munich.108660_IWEC.epw"
requiredUValuesPath = r"C:\Git\Work\FOR2363\Parametric-H-Shape-Office-Building-Model\src\Required-U-Values.csv"
directorScriptPath = "Director.gh"
reportScriptPath = "Reporting.gh"
simulationScriptPath = "Test.gh"
computeUrl = "http://localhost:6500/"

In [20]:
import compute_rhino3d
import compute_rhino3d.Grasshopper as gh
from rhino3dm import GeometryBase,File3dm,CommonObject,ObjectAttributes,Point3d
import json
from os.path import join
from numpy import float64, int32, bool_
compute_rhino3d.Util.authToken = ""
compute_rhino3d.Util.url = computeUrl

In [21]:
def callGrasshopper(path,parameters):
    trees = convertToTrees(parameters)
    return gh.EvaluateDefinition(path, trees)

def convertToTrees(parameters):
    trees = []
    if type(parameters) is not dict:
        return "Parameters have to be a dictionary"
    for kw in parameters:
        branches = parameters[kw]
        #empty branches will be skipped
        if branches:
            tree = gh.DataTree(kw)
            #check if branches is just item
            if type(branches) != list:
                tree.Append([0], [encode(branches)])
            else:
                #check if branches is list
                if type(branches[0]) != list:
                    tree.Append([0], [encode(item) for item in branches])
                else:
                    for i,branch in enumerate(branches):
                        tree.Append([i], [encode(item) for item in branch])
            trees.append(tree)
    return trees

def encode(value):
    if type(value).__module__.startswith('rhino3dm'):
        return json.dumps(value.Encode())
    else:
        return value

In [22]:
def encryptPathForCompute(path):
    return path.replace('\\','%').replace(':','$')

def getModelName(i):
    return "Simulation"+str(i).zfill(3)

def translateToIdf(i):
    modelName = getModelName(i)
    parameters = {
        'Honeybee-FilePath': encryptPathForCompute(join(outputDirectory, modelName + ".hbjson")),
        'EPW-FilePath': encryptPathForCompute(weatherFilePath),
        'Output-Directory': encryptPathForCompute(outputDirectory),
        'Output-Names':['Dummy'],
        'Report-Names':["AllSummary"],
        'SimulationMode':0,
        'AdditionalIDF':"!-Dummy",
        'Terrain': "City",
    }
    
    r =callGrasshopper(simulationScriptPath,{})
    return r
translateToIdf(2)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)